# HRDIC analysis for finding CP parameters

Find HRDIC analysis parameters then put into automated code at the end.

In [1]:
# Load third-party packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5
from defdap import quat
from defdap import ebsd
from defdap import hrdic
from defdap import defaults
import pandas as pd
import pickle
from skimage import io,color
from defdap import defaults
import HRDIC_vs_CP as hc

## HRDIC slip and strain analysis
With many data sets, makes it easier to include values in dicts.

In [29]:
#slip activaiton
#A3_zone1 = 8
crop_area = {'A3_zone1':[30, 30, 20, 20],
            'A3_zone2':[30,40,10,10],
            'A3_zone3':[30,35,10,10]
            }

dic_homog = {'A3_zone1':[(1765, 335), (738, 215), (1373, 1167), (1550, 1579), (1816, 1790), (1055, 664), (309, 1141), (263, 681)],
           'A3_zone2':[(1120, 115), (2089, 703), (583, 1117), (944, 1458), (150, 869), (1508, 871)],
           'A3_zone3':[(610, 529), (2083, 216), (1571, 2080), (224, 1958), (243, 986), (850,1394), (1204, 1599)],
           'B2_zone1':[(1867, 112), (688, 1112), (1378, 293), (391, 796), (2755, 1534), (2730, 1035), (1933, 1727)]}

ebsd_homog = {'A3_zone1':[(402, 352), (335, 343), (376, 410), (388, 439), (405, 452), (356, 373), (307, 407), (303, 376)],
           'A3_zone2':[(280, 100), (342, 141), (244, 169), (266, 192), (215, 151), (305, 153)],
           'A3_zone3': [(236, 200), (334, 177), (299, 310), (211, 304), (213, 232), (245, 253), (274, 276)],
           'B2_zone1':[(292, 131), (218, 198), (261, 143), (198,177), (356, 225), (354, 191), (303, 242)]}

dicFilePath ={'A3_zone1':"../DIC_data/A3/zone_1/take 3/",
             'A3_zone2':"../DIC_data/A3/zone_2/take3/",
             'A3_zone3':"../DIC_data/A3/zone_3/take 3/",
             'B2_zone1':"../DIC_data/B2/take 2/"}

EbsdFilePath={'A3_zone1':'../EBSD_data/CP large area/A3_zone1',
              'A3_zone2':'../EBSD_data/CP large area/A3_zone2',
              'A3_zone3':'../EBSD_data/CP large area/A3_zone3',
              'B2_zone1':'../EBSD_data/CP large area/B2_zone1'}

EbsdFlip = {'A3_zone1':False,
              'A3_zone2':False,
              'A3_zone3':False,
              'B2_zone1':True}

min_grain ={'A3_zone1':50,
           'A3_zone2':50,
           'A3_zone3':50,
           'B2_zone1':150}

In [30]:
step = '14' # two digits 

region = 'A3_zone3'
#dicFilePath = "../DIC_data/A3/zone_2"
DicMap = hrdic.Map(dicFilePath[region], "B000{}.txt".format(step))

Loaded DaVis 8.4.0 data (dimensions: 2160 x 2214 pixels, sub-window size: 12 x 12 pixels)


Optional mask to remove noisy points from data sets

In [ ]:
#shear_map = DicMap.eMaxShear
#mask = io.imread(dicFilePath[region] +'/02-1.bmp')

#plt.imshow(mask)

#shear_map[mask < 20 ] = np.nan 

Normal HRDIC analysis

In [32]:
DicMap.setPatternPath('01-1.BMP',1) 
DicMap.setCrop(xMin=crop_area[region][0], xMax=crop_area[region][1], yMin=crop_area[region][2], yMax=crop_area[region][3])
DicMap.setScale(micrometrePerPixel=40/2048)
np.mean(DicMap.crop(DicMap.e11))

0.060839165461511586

In [33]:
DicMap.plotMaxShear(plotColourBar=True, plotScaleBar=True, vmin=0, vmax=0.1)
#plt.savefig(output + '{}'.format(step),dpi=1600)

In [27]:
#EbsdFilePath = "../EBSD_data/CP large area/A3_zone2"
EbsdMap = ebsd.Map(EbsdFilePath[region]) #delete 'cubic'
if EbsdFlip[region]:
    EbsdMap.rotateData()


Starting loading EBSD data..
Unknown field in file with key 51. Assumming float32 data.
Loaded EBSD data (dimensions: 606 x 453 pixels, step size: 3.5 um)


In [28]:
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 2) #degrees
EbsdMap.findGrains(minGrainSize = min_grain[region]) #pixels
EbsdMap.calcGrainMisOri(calcAxis = False)
EbsdMap.calcAverageGrainSchmidFactors(loadVector=[1,0,0])

Finished building quaternion array (0:00:01) 
Finished finding grain boundaries (0:00:01) 
Finished finding grains (0:00:01) 
Finished calculating grain misorientations (0:00:04) 
Finished calculating grain average Schmid factors (0:00:00) 


Find the correct homogPoints and then edit the values at the top

In [ ]:
EbsdMap.plotHomog=EbsdMap.plotBandContrastMap

In [ ]:
DicMap.homogPoints = []
EbsdMap.homogPoints = []

In [ ]:
DicMap.setHomogPoint(display="pattern")
DicMap.setHomogPoint(vmax=0.04)
#DicMap.setHomogPoint(map_name = "pattern")
EbsdMap.setHomogPoint()

In [ ]:
print(DicMap.homogPoints)
print(EbsdMap.homogPoints)

In [10]:
#DicMap.homogPoints = [(1765, 335), (738, 215), (1373, 1167), (1550, 1579), (1816, 1790), (1055, 664), (309, 1141), (263, 681)] 

#EbsdMap.homogPoints = [(402, 352), (335, 343), (376, 410), (388, 439), (405, 452), (356, 373), (307, 407), (303, 376)] 

In [11]:
EbsdMap.homogPoints=ebsd_homog[region]
DicMap.homogPoints=dic_homog[region]

In [12]:
#DicMap.linkEbsdMap(EbsdMap, transformType='polynomial', order = 2)
DicMap.linkEbsdMap(EbsdMap, transformType='affine')

In [14]:
DicMap.plotMaxShear(plotGBs=True, dilateBoundaries=True, plotColourBar=True, plotScaleBar=True, vmin=0, vmax=0.01)
plt.tight_layout()
plt.savefig(output + 'DIC_step11',dpi = 1200)

In [15]:
DicMap.findGrains(algorithm='warp')
#DicMap.findGrains(algorithm='floodfill')

Finished finding grains (0:00:00) 


In [16]:
#EbsdMap.plotIPFMap([1,0,0], plotScaleBar=True,plotGB=True)

In [17]:
plt.figure(figsize=(3,2))
DicMap.plotGrainAvMaxShear(vmin=0,vmax=0.03,plotColourBar=True,plotScaleBar=True,plotGBs= True ,dilateBoundaries=True)
plt.tight_layout()
plt.savefig(output +'DIC_grain_step11',dpi=1200)#,dpi=1600,figsize=(3,2))

In [18]:
DicMap.locateGrainID(vmax=0.01)

In [19]:
EbsdMap.calcAverageGrainSchmidFactors(loadVector=np.array([1,0,0]))

Possible to save slip activity as a text file, in my case the data was in a different format so i needed to write into a spreadsheet.

In [21]:
DicMap.runGrainInspector(vmax=0.02,corrAngle=0)

In [67]:
int('a')

ValueError: invalid literal for int() with base 10: 'a'

## Slip activation 

Reimport slip data to ccombined and compare between regions

In [37]:
slip_data_frame = pd.read_csv('Slip_activiation_A3_zone1.csv',skiprows=5)
#slip_data_frame = pd.read_csv('Slip_activiation_A3_zone2.csv',skiprows=5)
#slip_data_frame = pd.read_csv('Slip_activiation_A3_zone3.csv',skiprows=5)

stress_at_step_a3 = np.load('exp_eng_stress_at_image_a3.npy')
stress_at_step_b2 = np.load('exp_eng_stress_at_image_b2.npy')

num_grains_a3_zone1 = 37
num_grains_a3_zone2 = 26
num_grains_a3_zone3 = 25

num_grains_combined = np.sum([num_grains_a3_zone1,num_grains_a3_zone2,num_grains_a3_zone3])/100
#num_grains_combined
#slip_data_frame

In [36]:
stress_at_step_a3

array([  1.9047619 ,   5.71428571,  21.52380952,  63.23809524,
       276.38095238, 361.14285714, 446.0952381 , 531.61904762,
       619.04761905, 706.0952381 , 786.85714286, 856.19047619,
       897.9047619 , 918.66666667, 925.71428571, 936.19047619,
       942.0952381 , 947.42857143, 950.0952381 , 954.0952381 ])

In [3]:
step_grain_dict=hc.create_step_grain_dict(slip_data_frame)
slip_activation=hc.calculate_slip_activation(step_grain_dict)

In [4]:
sorted_keys = sorted(slip_activation.keys())
sorted_slip_activation = {key: slip_activation[key] for key in sorted_keys}

Save into dict format for each region and then reimport together

file_name = "A3_zone1_slip_activation.pickle".format(region)
with open(file_name, "wb") as pickle_file:
    pickle.dump(DIC_data, pickle_file)

In [54]:
#slip_activation

In [38]:
file_name_1 = "A3_zone1_slip_activation.pickle"
with open(file_name_1, "rb") as pickle_file:
    slip_activation_A3_zone1 = pickle.load(pickle_file)

file_name_2 = "A3_zone2_slip_activation.pickle"
with open(file_name_2, "rb") as pickle_file:
    slip_activation_A3_zone2 = pickle.load(pickle_file)

file_name_3 = "A3_zone3_slip_activation.pickle"
with open(file_name_3, "rb") as pickle_file:
    slip_activation_A3_zone3 = pickle.load(pickle_file)
# Now, 'loaded_dict' contains the dictionary that was saved in the Pickle file
#print(slip_activation_A3_zone3)

In [39]:
slip_activation_zones = [slip_activation_A3_zone1]#, slip_activation_A3_zone2, slip_activation_A3_zone3]
slip_activation_combined = hc.combine_slip_activation(slip_activation_zones)
#slip_activation_combined = slip_activation_A3_zone1
#slip_activation_combined

In [40]:
sorted_keys = sorted(slip_activation_combined.keys())
sorted_slip_activation_combined = {key: slip_activation_combined[key] for key in sorted_keys}
#sorted_slip_activation_combined

In [41]:
exp_slip_system_counts = hc.HRDIC_count_slip_systems(sorted_slip_activation_combined)
exp_slip_system_counts

{'Basal': [2, 1, 1, 0, 0, 1, 3],
 'Prismatic': [0, 2, 2, 3, 7, 1, 0],
 'Pyramidal <a>': [0, 3, 2, 5, 7, 1, 1],
 'Pyramidal <a+c>': [0, 0, 3, 1, 3, 1, 0],
 'ambiguous': [4, 6, 1, 2, 4, 2, 3]}

In [1]:
def calculate_cumulative_sum(original_dict):
    result_dict = {}
    for system, values in original_dict.items():
        cumulative_sum = []
        running_total = 0
        for value in values:
            running_total += value
            cumulative_sum.append(running_total)
        result_dict[system] = cumulative_sum
    return result_dict

# Example data
original_dict = {
    'Basal': [2, 1, 1, 0, 0, 1, 3],
    'Prismatic': [0, 2, 2, 3, 7, 1, 0],
    'Pyramidal <a>': [0, 3, 2, 5, 7, 1, 1],
    'Pyramidal <a+c>': [0, 0, 3, 1, 3, 1, 0],
    'ambiguous': [4, 6, 1, 2, 4, 2, 3]
}

result_dict = calculate_cumulative_sum(original_dict)
print(result_dict)


{'Basal': [2, 3, 4, 4, 4, 5, 8], 'Prismatic': [0, 2, 4, 7, 14, 15, 15], 'Pyramidal <a>': [0, 3, 5, 10, 17, 18, 19], 'Pyramidal <a+c>': [0, 0, 3, 4, 7, 8, 8], 'ambiguous': [4, 10, 11, 13, 17, 19, 22]}


In [42]:
steps = list(sorted_slip_activation_combined.keys())
categories = list(exp_slip_system_counts.keys())
bottom = [0] * len(steps)

plt.figure()
for category in categories:
    plt.bar(steps, exp_slip_system_counts[category], label=category, bottom=bottom)
    bottom = [bottom[i] + exp_slip_system_counts[category][i] for i in range(len(steps))]

plt.xlabel('Step')
plt.ylabel('Frequency of system')
plt.title('Incremental slip activation for combined region')
plt.legend()
plt.show()

In [43]:
exp_cumulative_counts_dict = hc.calculate_cumulative_counts(sorted_slip_activation_combined)
exp_cumulative_counts=hc.flatten_nested_dict(exp_cumulative_counts_dict)
#exp_cumulative_counts

In [50]:
# Plot a stacked bar chart
steps = list(exp_cumulative_counts_dict.keys())
categories = list(exp_cumulative_counts.keys())
bottom = [0] * len(steps)

plt.figure()
for category in categories:
    plt.bar(steps, exp_cumulative_counts[category], label=category, bottom=bottom)
    bottom = [bottom[i] + exp_cumulative_counts[category][i] for i in range(len(steps))]

plt.xlabel('Step')
plt.ylabel('Frequency')
plt.title('Incremental slip activation for combined region')
plt.legend()
plt.show()

In [45]:
fig, (ax1, ax2) = plt.subplots(1, 2)#,dpi=800, figsize=(12, 6))

# Plot the first graph on the first subplot
steps = list(sorted_slip_activation_combined.keys())
categories = list(exp_slip_system_counts.keys())
bottom = [0] * len(steps)

for category in categories:
    ax1.bar(steps, exp_slip_system_counts[category], label=category, bottom=bottom)
    bottom = [bottom[i] + exp_slip_system_counts[category][i] for i in range(len(steps))]

ax1.set_xlabel('Step')
ax1.set_ylabel('Frequency of grians')
ax1.set_title('Incremental slip activation for combined region')
ax1.legend()

# Plot the second graph on the second subplot
bottom = [0] * len(steps)

for category in categories:
    ax2.bar(steps, exp_cumulative_counts[category], label=category, bottom=bottom)
    bottom = [bottom[i] + exp_cumulative_counts[category][i] for i in range(len(steps))]

ax2.set_xlabel('Step')
ax2.set_ylabel('Frequency of grians')
ax2.set_title('Accumulated slip activation for combined region')
ax2.legend()

# Adjust layout
plt.tight_layout()

# Show plots
plt.show()
#plt.savefig(output+'/slip_activation_exp.png')

In [46]:
a3_stress_at_step = []
for i in range(len(stress_at_step_a3)):
    a3_stress_at_step.append(int(stress_at_step_a3[i]))
#a3_stress_at_step

In [49]:
fig, (ax1, ax2) = plt.subplots(1, 2)#,dpi=800, figsize=(12, 6))

# Plot the first graph on the first subplot
steps = list(sorted_slip_activation_combined.keys())
categories = list(exp_slip_system_counts.keys())
bottom = [0] * len(steps)

for category in categories:
    ax1.bar(steps, exp_slip_system_counts[category], label=category, bottom=bottom)
    bottom = [bottom[i] + exp_slip_system_counts[category][i] for i in range(len(steps))]

ax1.set_xlabel('Eng stress (MPa)')
ax1.set_ylabel('Frequency of system')
ax1.set_title('Incremental slip activation for combined region')
ax1.legend()

# Set custom x-ticks for the first subplot
stress_ticks = a3_stress_at_step[5:14] # Define your custom ticks
ax1.set_xticks(np.arange(len(custom_ticks))+6)  # Set the tick positions
ax1.set_xticklabels(stress_ticks)#, rotation=90)  # Set the tick labels
# Plot the second graph on the second subplot
bottom = [0] * len(steps)

for category in categories:
    ax2.bar(steps, exp_cumulative_counts[category], label=category, bottom=bottom)
    bottom = [bottom[i] + exp_cumulative_counts[category][i] for i in range(len(steps))]

ax2.set_xlabel('Eng Stress (MPa)')
ax2.set_ylabel('Frequency of system')
ax2.set_title('Accumulated slip activation for combined region')
ax2.legend()

stress_ticks = a3_stress_at_step[5:14] # Define your custom ticks
ax2.set_xticks(np.arange(len(custom_ticks))+6)  # Set the tick positions
ax2.set_xticklabels(stress_ticks)#, rotation=90)  # Set the tick labels
# Plot the second graph on the second subplot
# Adjust layout
plt.tight_layout()

# Show plots
#plt.show()
#plt.savefig(output + 'Slip_activation_exp.png')

In [ ]:
#from defdap import defaults
#defaults['crystal_ortho_conv'] = 'hkl'

In [45]:
grainOris = [grain.ebsdGrain.refOri for grain in DicMap]
plot = quat.Quat.plotIPF(grainOris, direction=[1,0,0], symGroup='hexagonal', marker='o')

In [46]:
slip_activation = slip_activation_A3_zone2
#slip_activation

In [48]:
for key, value in slip_activation.items():
    # Check if the value is a dictionary
    if isinstance(value, dict):
        # Iterate through the inner dictionary
        for inner_key, inner_value in value.items():
            # Check if the value is a list with at least one element
            if isinstance(inner_value, list) and len(inner_value) >= 1:
                # Modify the list to keep only the first element (string)
                slip_activation[key][inner_key] = inner_value[0][0]

In [31]:
new_dict = {}

# Iterate over the steps in reverse order
for step in range(max(slip_activation.keys()), -1, -1):
    new_grains = {}
    for prev_step in range(step + 1):
        for grain, value in slip_activation[prev_step].items():
            new_grains.setdefault(grain, value)  # Corrected order
    new_dict[step] = new_grains

sorted_dict = {k: v for k, v in sorted(new_dict.items(), key=lambda item: item[0], reverse=False)}
sorted_nested_dict = {step: {k: v for k, v in sorted(sub_dict.items())} for step, sub_dict in sorted_dict.items()}

#print(sorted_nested_dict)


new_dict = {}

# Iterate over the steps in reverse order
for step in range(max(original_dict.keys()), -1, -1):
    new_grains = {}
    for prev_step in range(step + 1):
        for grain, value in original_dict[prev_step].items():
            if grain not in new_grains or new_grains[grain] == 'ambiguous':
                new_grains[grain] = value
    new_dict[step] = new_grains

sorted_dict = {k: v for k, v in sorted(new_dict.items(), key=lambda item: item[0], reverse=False)}
sorted_nested_dict = {step: {k: v for k, v in sorted(sub_dict.items())} for step, sub_dict in sorted_dict.items()}

print(sorted_nested_dict)


In [32]:
#sorted_nested_dict
#sorted_nested_dict[int(steps)]

In [34]:
output ='../results/IPF/'

In [37]:
f, ax = plt.subplots(figsize=(4,3))

markersize=50
quat.Quat.plotIPF(allgrains, [1,0,0], 'hexagonal', color='0.8', marker='o', s=markersize-20, ax=ax,label='All grains')  
quat.Quat.plotIPF(baorilist, [1,0,0], 'hexagonal' , marker='h', s=markersize, ax=ax,label='Basal') # c='b'
quat.Quat.plotIPF(prorilist, [1,0,0], 'hexagonal', marker='s', s=markersize, ax=ax,label='Prismatic') #  c='g',
quat.Quat.plotIPF(py_aorilist, [1,0,0], 'hexagonal',  marker='^', s=markersize, ax=ax,label='Pyramidal <a>') #  c='r',
quat.Quat.plotIPF(py_acorilist, [1,0,0], 'hexagonal',  marker='d', s=markersize, ax=ax,label='Pyramidal <a+c>') # 
quat.Quat.plotIPF(ambiglist, [1,0,0], 'hexagonal',  marker='*', s=markersize, ax=ax,label='Ambiguous') #c='m', 

plt.legend()
output ='../results/IPF/'
f.tight_layout()
f.savefig(output + 'A3_zone2_step{}_ipf.png'.format(str(step)), dpi=1200)

new_dict = {}

# Iterate over the steps in reverse order
for step in range(max(original_dict.keys()), -1, -1):
    new_grains = {}
    for prev_step in range(step + 1):
        for grain, value in original_dict[prev_step].items():
            new_grains.setdefault(value, grain)
    new_dict[step] = new_grains
sorted_dict = {k: v for k, v in sorted(new_dict.items(), key=lambda item: item[0], reverse=False)}

print(sorted_dict[14])

#print(new_dict)

prorilist = []; pyorilist = []; baorilist = []; multorilist = [];  allgrains=[]; 

for idx, grain in enumerate(DicMap):
    
    try:
        system = grain.groupsList[0][2]
    except IndexError:
        continue
        
    allgrains.append(grain.ebsdGrain.refOri)
        
    
    
    if len(system)>1 or system==[]:
        i=1
    else: 
        if system == [0]:
            baorilist.append(grain.ebsdGrain.refOri)
        if system == [1] or system == [2] or system == [3]:
            prorilist.append(grain.ebsdGrain.refOri)
        if system == [4] or system == [5] or system == [6] or system == [7] or system == [8] or system == [9]:
            pyorilist.append(grain.ebsdGrain.refOri)


f, ax = plt.subplots(figsize=(4,3))

markersize=40
quat.Quat.plotIPF(allgrains, [1,0,0], 'hexagonal', c='b', marker='o', s=markersize, ax=ax,label='All grains')  
quat.Quat.plotIPF(allgrains, [1,0,0], 'hexagonal', c='0.9', marker='o', s=markersize, ax=ax,label='All grains')  
quat.Quat.plotIPF(multorilist, [1,0,0], 'hexagonal', c='0.4', marker='o', s=markersize, ax=ax,label='Multi slip')
quat.Quat.plotIPF(prorilist, [1,0,0], 'hexagonal', c='g', marker='s', s=markersize, ax=ax,label='Prismatic only')
quat.Quat.plotIPF(pyorilist, [1,0,0], 'hexagonal', c='r', marker='^', s=markersize, ax=ax,label='Pyramidal only')
quat.Quat.plotIPF(baorilist, [1,0,0], 'hexagonal', c='b', marker='h', s=markersize, ax=ax,label='Basal only')
  
# quat.Quat.plotIPF(allgrains, [1,0,0], 'hexagonal', marker='o', s=markersize, ax=ax)  
# quat.Quat.plotIPF(multorilist, [1,0,0], 'hexagonal', marker='o', s=markersize, ax=ax)  
# quat.Quat.plotIPF(prorilist, [1,0,0], 'hexagonal', marker='o', s=markersize, ax=ax)
# quat.Quat.plotIPF(pyorilist, [1,0,0], 'hexagonal', marker='o', s=markersize, ax=ax)
# quat.Quat.plotIPF(baorilist, [1,0,0], 'hexagonal', marker='o', s=markersize, ax=ax)
plt.legend()

f.tight_layout()
#f.savefig(output + 'B2_zone1_step13.png', dpi=1200)

EbsdFilePath = "../EBSD_data/DIC actual area/A3_zone1"
EbsdMap = ebsd.Map(EbsdFilePath) #delete 'cubic'
#EbsdMap.rotateData()
EbsdMap.findGrains()

allgrains = []
for grain in EbsdMap:
    grain.calcAverageOri()
    allgrains.append(grain.refOri)
#np.save("a3_z3_grainAvOris.npy",allgrains)
##### Plot all the grain orientations in the map
quat.Quat.plotIPF(allgrains, [1, 0, 0], EbsdMap.crystalSym, marker='o', s=10)
plt.tight_layout()

In [135]:
from orix import plot
ckey_m3m = plot.IPFColorKeyTSL("m-3m")
fig = ckey_m3m.plot(return_figure=True)  # Returns a Matplotlib figure
fig.savefig("ckey_m3m.png")

ModuleNotFoundError: No module named 'orix'

In [ ]:
grain_ID = []
DIC_ESS = []
grain_size = []
DIC_e11 = []
for k in range(0,len(DicMap.grainList)):
    grain_ID.append(DicMap[k].ebsdGrain.grainID)
    DIC_ESS.append(np.mean(DicMap[k].maxShearList))
    grain_size.append(len(DicMap[k].maxShearList))
    #DIC_e11.append(DicMap.calcGrainAv(DicMap.crop(DicMap.e11)))
DIC_e11=(DicMap.calcGrainAv(DicMap.crop(DicMap.e11)))
DIC_e22=(DicMap.calcGrainAv(DicMap.crop(DicMap.e22)))
DIC_e12=(DicMap.calcGrainAv(DicMap.crop(DicMap.e12)))
DIC_shear_strain = (( np.mean(DIC_e11) - np.mean(DIC_e22) )**2/2 + np.mean(DIC_e12)**2  )**0.5 
DIC_shear_strain
#print(EBSD_DIC_grains)
#print(DIC_grain_strain)


In [ ]:
std_shear_strain = calculate_std_shear_strain(DIC_e11, DIC_e22, DIC_e12)
print("Standard Deviation of Shear Strain:", std_shear_strain)

In [ ]:
# Define the length of the array 'L'
L = 10  # Replace with your desired length

# Replace 'DIC_std_shear_strain' with your actual value
DIC_std_shear_strain = 0.002  # Replace with your value

# Create an array of length 'L' with the same value as 'DIC_std_shear_strain'
array_of_values = np.full((L,), DIC_std_shear_strain)
array_of_values

In [ ]:
step_11 = np.column_stack((grain_ID, grain_size, DIC_ESS,DIC_e11,DIC_e22,DIC_e22))
df=pd.DataFrame(step_11)
df

In [ ]:
DicMap.calcGrainAv(DicMap.crop(DicMap.e11))

In [ ]:
result = np.full((len(DicMap.grainList,),), std_shear_strain)
result

In [1]:
#IMPORT PACKAGES
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5
from defdap import quat
from defdap import ebsd
from defdap import hrdic
import pandas as pd
import pickle

##################################################################################################################
#SAMPLE PARAMETERS
region='A3_zone2' #change file path
DIC_data = {}
resolution = 40/2048
DIC_mean_shear_strain = []
DIC_std_shear_strain  = []
output ="../results/{}/".format(region)
plot_maps = True
#slip activaiton
#A3_zone1 = 8
crop_area = {'A3_zone1':[30, 30, 20, 20],
            'A3_zone2':[30,40,10,10],
            'A3_zone3':[30,35,10,10]
            }

dic_homog = {'A3_zone1':[(1765, 335), (738, 215), (1373, 1167), (1550, 1579), (1816, 1790), (1055, 664), (309, 1141), (263, 681)],
           'A3_zone2':[(1120, 115), (2089, 703), (583, 1117), (944, 1458), (150, 869), (1508, 871)],
           'A3_zone3':[(610, 529), (2083, 216), (1571, 2080), (224, 1958), (243, 986), (850,1394), (1204, 1599)],
           'B2_zone1':[(1867, 112), (688, 1112), (1378, 293), (391, 796), (2755, 1534), (2730, 1035), (1933, 1727)]}

ebsd_homog = {'A3_zone1':[(402, 352), (335, 343), (376, 410), (388, 439), (405, 452), (356, 373), (307, 407), (303, 376)],
           'A3_zone2':[(280, 100), (342, 141), (244, 169), (266, 192), (215, 151), (305, 153)],
           'A3_zone3': [(236, 200), (334, 177), (299, 310), (211, 304), (213, 232), (245, 253), (274, 276)],
           'B2_zone1':[(292, 131), (218, 198), (261, 143), (198,177), (356, 225), (354, 191), (303, 242)]}

dicFilePath ={'A3_zone1':"../DIC_data/A3/zone_1/take 3/",
             'A3_zone2':"../DIC_data/A3/zone_2/take3/",
             'A3_zone3':"../DIC_data/A3/zone_3/take 3/",
             'B2_zone1':"../DIC_data/B2/take 2/"}

EbsdFilePath={'A3_zone1':'../EBSD_data/CP large area/A3_zone1',
              'A3_zone2':'../EBSD_data/CP large area/A3_zone2',
              'A3_zone3':'../EBSD_data/CP large area/A3_zone3',
              'B2_zone1':'../EBSD_data/CP large area/B2_zone1'}

EbsdFlip = {'A3_zone1':False,
              'A3_zone2':False,
              'A3_zone3':False,
              'B2_zone1':True}

min_grain ={'A3_zone1':50,
           'A3_zone2':50,
           'A3_zone3':50,
           'B2_zone1':150}
##################################################################################################################
#IMPORT DIC DATA
for step in np.arange(1,15): #index from 1
    if step <10:
        DicMap = hrdic.Map(dicFilePath[region], "B0000{}.txt".format(step))
    else:
        DicMap = hrdic.Map(dicFilePath[region], "B000{}.txt".format(step))
    
###################################################################################################################
#LINK DIC AND EBSD 
    DicMap.setPatternPath('01-1.BMP',1) 
    DicMap.setCrop(xMin=crop_area[region][0], xMax=crop_area[region][1], yMin=crop_area[region][2], yMax=crop_area[region][3])
    DicMap.setScale(micrometrePerPixel=resolution)

    EbsdMap = ebsd.Map(EbsdFilePath[region]) #delete 'cubic'
    
    if EbsdFlip[region]:
        EbsdMap.rotateData()

    EbsdMap.buildQuatArray()
    EbsdMap.findBoundaries(boundDef = 2) #degrees
    EbsdMap.findGrains(minGrainSize = min_grain[region]) #pixels
    EbsdMap.calcGrainMisOri(calcAxis = False)
    EbsdMap.calcAverageGrainSchmidFactors(loadVector=[1,0,0])
    
    DicMap.homogPoints = dic_homog[region]
    EbsdMap.homogPoints = ebsd_homog[region]

    #DicMap.linkEbsdMap(EbsdMap, transformType='polynomial', order = 2)
    DicMap.linkEbsdMap(EbsdMap, transformType='affine')

    DicMap.findGrains(algorithm='warp')
    EbsdMap.calcAverageGrainSchmidFactors(loadVector=np.array([1,0,0]))
#########################################################################################################################
#PLOT MAPS
    if plot_maps: 
        DicMap.plotGrainAvMaxShear(vmin=0,vmax=0.015,plotColourBar=True,plotScaleBar=True,plotGBs= True ,dilateBoundaries=True)
        plt.tight_layout()
        plt.savefig(output + 'Grain_avg_ESS_DIC_{}'.format(step),dpi=1600)
        plt.close()
        DicMap.plotMaxShear(plotGBs=True, dilateBoundaries=True, plotColourBar=True, plotScaleBar=True, vmin=0, vmax=0.015)
        plt.tight_layout()
        plt.savefig(output + 'ESS_DIC_{}'.format(step),dpi=1600)
        plt.close()
#######################################################################################################################
#ANALYSIS STRAINS
    grain_ID = []
    DIC_ESS = []
    grain_size = []
    DIC_e11 =[]
    DIC_e22 =[]
    DIC_e12 =[]
    for k in range(0,len(DicMap.grainList)):
        grain_ID.append(DicMap[k].ebsdGrain.grainID)
        grain_size.append(len(DicMap[k].maxShearList)*resolution)
    
    DIC_e11=(DicMap.calcGrainAv(DicMap.crop(DicMap.e11)))
    DIC_e22=(DicMap.calcGrainAv(DicMap.crop(DicMap.e22)))
    DIC_e12=(DicMap.calcGrainAv(DicMap.crop(DicMap.e12)))
    shear_strain=[]
    for i in range(len(DIC_e11)):    
        shear_strain.append( (( (DIC_e11[i]) - (DIC_e22[i]) )**2/2 + (DIC_e12[i])**2  )**0.5 )
    DIC_mean_shear_strain = np.mean(shear_strain)
    DIC_std_shear_strain = np.std(shear_strain)
#####################################################################################################################
#EXPORT DATA
    
    #DIC_data['step_{}'.format(str(step))]=np.column_stack((grain_ID, grain_size, DIC_e11,shear_strain))
    DIC_data['step_{}'.format(str(step))] = {'grain_ID':grain_ID,'grain_size ($\mu m^2$)':grain_size,'DIC_e11':DIC_e11,'DIC_shear_strain':shear_strain}
file_name = "{}_strain_DIC.pickle".format(region)
with open(file_name, "wb") as pickle_file:
    pickle.dump(DIC_data, pickle_file)

Loaded DaVis 8.4.0 data (dimensions: 2169 x 2128 pixels, sub-window size: 12 x 12 pixels)
Starting loading EBSD data..
Unknown field in file with key 51. Assumming float32 data.
Loaded EBSD data (dimensions: 554 x 313 pixels, step size: 3.5 um)
Finished building quaternion array (0:00:00) 
Finished finding grain boundaries (0:00:00) 
Finished finding grains (0:00:01) 
Finished calculating grain misorientations (0:00:02) 
Finished calculating grain average Schmid factors (0:00:00) 
Finished finding grains (0:00:00) 
Finished calculating grain average Schmid factors (0:00:00) 
Loaded DaVis 8.4.0 data (dimensions: 2169 x 2128 pixels, sub-window size: 12 x 12 pixels)
Starting loading EBSD data..
Unknown field in file with key 51. Assumming float32 data.
Loaded EBSD data (dimensions: 554 x 313 pixels, step size: 3.5 um)
Finished building quaternion array (0:00:00) 
Finished finding grain boundaries (0:00:00) 
Finished finding grains (0:00:00) 
Finished calculating grain misorientations (0:0

In [ ]:
file_name = "A3_zone2_strain_DIC.pickle"
with open(file_name, "rb") as pickle_file:
    strain_comparison = pickle.load(pickle_file)

# Now, 'loaded_dict' contains the dictionary that was saved in the Pickle file
print(strain_comparison)